## Teste 1
##### Programação grosseira em busca dos melhores dados e dos melhores modelos

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from warnings import simplefilter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 8)
#sns.set_style('whitegrid')
sns.set_style('white')
sns.set_context('poster')

In [53]:
df = pd.read_csv('datasets/dataset_cler.csv', sep=";", encoding='utf-16')
df.shape

(1642, 115)

In [54]:
df.head(2)

,ampl_9,desv_amp_9,body_9,body_per_9,pavio_sup_9,pavio_sup_per_9,pavio_inf_9,pavio_inf_per_9,high_dist_9,low_dist_9,...,pavio_inf_0,pavio_inf_per_0,high_dist_0,low_dist_0,type_0,atr,qt_bars,oper,take,status
0,70.0,1.17,50.0,71.43,20.0,28.57,0.0,0.0,25.0,35.0,...,5.0,11.11,5.0,5.0,down,58.21,5,Sell,20.0,Gain
1,50.0,0.63,25.0,50.00,10.0,20.00,15.0,30.0,5.0,35.0,...,20.0,30.77,35.0,25.0,up,66.07,6,Sell,-40.0,Loss


In [55]:
df['status'].value_counts()

Gain    1099
Loss     543
Name: status, dtype: int64

In [56]:
df.drop(df.loc[(df['status']!='Loss') & (df['status']!='Gain')].index.values, inplace=True)

In [57]:
df['status'].value_counts()

Gain    1099
Loss     543
Name: status, dtype: int64

In [58]:
#train_features = df.columns.difference(['take', 'status', 'oper'])
X = df.drop(['take','status','oper'], axis=1) #df[train_features]
y = df['status']

x_columns = X.columns

In [59]:
index_features_category = []
for index, col in enumerate(X.columns):
    if X.dtypes[index]=='object':
        index_features_category.append(index)
index_features_category[:10]

[10, 21, 32, 43, 54, 65, 76, 87, 98, 109]

In [60]:
encoder = LabelEncoder()
encoder.fit(['inside', 'up', 'down'])
columns_categorical = X.select_dtypes(include=['object']).columns
for col_cat in columns_categorical:
    X[col_cat] = encoder.transform(X[col_cat])

ohe = OneHotEncoder(categorical_features=index_features_category)
X = ohe.fit_transform(X).toarray()

In [61]:
def fit(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return y_pred 

def fitAll(X_train, X_test, y_train, y_test, algorithms):
    similaridade = []
    for item in algorithms:
        name = item[0]
        algorithm = item[1]
        
        algorithm.fit(X_train, y_train)
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #print(name, ' => ', accuracy)
        similaridade.append((accuracy, name))
        
    similaridade.sort()
    similaridade.reverse()
    
    for j in similaridade:
        print(j)

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [62]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

standar = StandardScaler()
standar.fit(X)
X = standar.fit_transform(X)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [64]:
algorithms = []

algorithms.append(['DecisionTreeClassifier', DecisionTreeClassifier(random_state=0)])
algorithms.append(['RandomForestClassifier', RandomForestClassifier(n_estimators=10, random_state=0)])
algorithms.append(['GaussianNB', GaussianNB()])
algorithms.append(['RidgeClassifier', RidgeClassifier()])
algorithms.append(['MLPClassifier', MLPClassifier(random_state=True, max_iter=1000, learning_rate_init=0.0001, hidden_layer_sizes=(89))])
algorithms.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
algorithms.append(['SVC', SVC(gamma='auto')])
algorithms.append(['KNeighborsClassifier', KNeighborsClassifier()])
algorithms.append(['LogisticRegression', LogisticRegression()])

fitAll(X_train, X_test, y_train, y_test, algorithms)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(0.6835699797160243, 'SVC')
(0.665314401622718, 'LogisticRegression')
(0.6511156186612576, 'RidgeClassifier')
(0.6490872210953347, 'RandomForestClassifier')
(0.6267748478701826, 'GradientBoostingClassifier')
(0.6227180527383367, 'KNeighborsClassifier')
(0.6125760649087221, 'MLPClassifier')
(0.6085192697768763, 'GaussianNB')
(0.5456389452332657, 'DecisionTreeClassifier')


test = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000)
y_pred = fit(X_train, X_test, y_train, y_test, test)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

In [65]:
y_test[300:301]

701    Gain
Name: status, dtype: object

In [66]:
X_test[300]

array([0.10447761, 0.08074534, 0.01694915, 0.1111    , 0.24      ,
       0.73684792, 0.08333333, 0.24688889, 0.00806452, 0.01652893,
       0.        , 0.06818182, 0.16720257, 0.01801802, 0.1818    ,
       0.17391304, 0.39155718, 0.2       , 0.48944648, 0.        ,
       0.03225806, 1.        , 0.2       , 0.18618989, 0.22666667,
       0.8947    , 0.08695652, 0.11232   , 0.        , 0.        ,
       0.18987342, 0.10447761, 1.        , 0.35869565, 0.17325228,
       0.38983051, 0.6571    , 0.11428571, 0.12192   , 0.17391304,
       0.26375909, 0.03      , 0.13978495, 0.        , 0.35714286,
       0.07258065, 0.21818182, 0.4444    , 0.15625   , 0.21165714,
       0.25641026, 0.39888003, 0.26829268, 0.05555556, 1.        ,
       0.1409396 , 0.05097087, 0.109375  , 0.6087    , 0.10714286,
       0.14042645, 0.22222222, 0.28698713, 0.01675978, 0.00485437,
       0.        , 0.17777778, 0.11208406, 0.07142857, 0.3158    ,
       0.22580645, 0.42982149, 0.25      , 0.37319783, 0.05925

In [67]:
#print('Positive/Positive Ok', conf_matrix[0][0])
#print('Positive/Negative Er', conf_matrix[0][1])
#print('Negative/Negative Ok', conf_matrix[1][0])
#print('Negative/Positive Ok', conf_matrix[1][1])

In [68]:
clf_importances = dict(zip(x_columns, clf_ran.feature_importances_))

NameError: name 'clf_ran' is not defined

In [ ]:
clf_ran.feature_importances_[0:10], len(clf_ran.feature_importances_)

index = np.arange(len(x_columns[0:64]))
plt.bar(index, clf_ran.feature_importances_[0:64])
plt.xticks(index, x_columns, rotation='vertical')
plt.title('clf_ran')
plt.show()

index = np.arange(len(x_columns[64:128]))
plt.bar(index, clf_ran.feature_importances_[64:128])
plt.xticks(index, x_columns, rotation='vertical')
plt.title('clf_ran')
plt.show()